In [70]:
import pandas as pd
import numpy as np
df = pd.read_csv('C:/Users/Ruslan/Documents/ИТ/CSVf/main_task.csv')

# Начинаем преобразования нескольких имеющихся столбцов, чтобы получить дополнительные полезные числовые признаки
# 1)Меняем Restaurant_ID, оставляя лишь номер в числовом формате

IDR = []
for i in range(0,len(df['Restaurant_id'])):
    IDR.append(int(df['Restaurant_id'][i][3:]))   
df['Restaurant_id'] = IDR

# 2)В Price Range заменим на значки $ на 100, 1000, 10000 
# В случае отсутствия значения, проставляем 626(это приблизительный вариант средней. 
# Иначе проставил бы 500, как наиболее распространённое значение)
PR = []
for i in range(0,len(df['Price Range'])):
    if df['Price Range'][i] == '$':
        PR.append(5)
    elif df['Price Range'][i] == '$$ - $$$':
        PR.append(500)
    elif df['Price Range'][i] == '$$$$':
        PR.append(5000)
    else:
        PR.append(626)
df['Price Range'] = PR

# 3) Из Cousine Style извлекаем  количество кухонь и полезные текстовые признаки 

SS = []
for i in range(0,len(df['Cuisine Style'])):
    m  = df['Cuisine Style'][i]
    if type(m) == str:
        m = m.replace("[", "").replace("]", "").replace("'", "").replace(" ", "").split(',')
        SS.append(m)
    else:
        SS.append('Neutral')

# Убираем  пробелы в начале названий кухонь, если таковые имеются.Легче потом будет заменить конкретные названия с уместными пробелами, благо их немного

df['Cuisine Style'] = SS

SSL = []
for i in range(0,len(SS)):
    SSL.append(len(SS[i]))
df['QuaCu'] = SSL

# На всякий случай формируем словарь названия кухонь и их количества во всех ресторанах 
CuDict = {}
for cs in range(0,len(SS)):
    if SS[cs] == 'Neutral':
        if SS[cs] in CuDict.keys():
            CuDict[SS[cs]] = CuDict[SS[cs]] + 1
        else:
            CuDict[SS[cs]] = 1
    else:
        for i in range(0,len(SS[cs])):
            if SS[cs][i] in CuDict.keys():
                CuDict[SS[cs][i]] = CuDict[SS[cs][i]] + 1
            else:
                CuDict[SS[cs][i]] = 1
# В дальнейшем предполагалось использовать его для некой величины зависимой от частоты кухонь, но первичный анализ показал, что это не улучшает MAE

In [1]:
# Добавляем дополнительные столбцы

In [71]:
#Добавим численность населения по имеющимся в базе городам. Данные в млн.человек. Источник:интернет, преимущественно Wikipedia.

df['City'].value_counts() # Нужно было чтобы получить список городов.
Cty = ['London','Paris','Madrid','Barcelona','Berlin','Milan','Rome','Prague','Lisbon','Vienna','Amsterdam','Brussels','Hamburg','Munich'
,'Lyon','Stockholm','Budapest','Warsaw','Dublin','Copenhagen','Athens','Edinburgh','Zurich','Oporto','Geneva','Krakow','Oslo','Helsinki','Bratislava','Luxembourg','Ljubljana'
]
Popul = {}

Popul['London'] = 8.4
Popul['Paris'] = 2.2
Popul['Madrid'] = 3.3
Popul['Barcelona'] = 1.6
Popul['Berlin'] = 3.6
Popul['Milan'] = 1.4
Popul['Rome'] = 2.9
Popul['Prague'] = 1.3
Popul['Lisbon'] = 0.5
Popul['Vienna'] = 1.9
Popul['Amsterdam'] = 0.8
Popul['Brussels'] = 0.2
Popul['Hamburg'] = 1.8
Popul['Munich'] = 1.5
Popul['Lyon'] = 0.5
Popul['Stockholm'] = 1.0
Popul['Budapest'] = 1.8
Popul['Warsaw'] = 1.7
Popul['Dublin'] = 1.4
Popul['Copenhagen'] = 0.6
Popul['Athens'] = 0.7
Popul['Edinburgh'] = 0.5
Popul['Zurich'] = 0.4
Popul['Oporto'] = 0.2
Popul['Geneva'] = 0.5
Popul['Krakow'] = 0.8
Popul['Oslo'] = 0.7
Popul['Helsinki'] = 0.6
Popul['Bratislava'] = 0.4
Popul['Luxembourg'] = 0.6
Popul['Ljubljana'] = 0.3

CitPop= []
for i in range(0,len(df['City'])):
    CitPop.append(Popul[df['City'][i]])
CitPop
df['CitPop'] = CitPop

In [72]:
# Добавляем показатель отношения числа отзывов по ресторану к численности населения (в млн.человек)
df['Rev/Pop'] = df['Number of Reviews']/df['CitPop']

In [73]:
# Добавляем показатель отношения числа отзывов по городу нахождения ресторана
DDF = df.groupby('City').sum()
NR = DDF['Number of Reviews']
CitRevNum = []
for i in range(0,len(df['City'])):
    CitRevNum.append(NR[df['City'][i]])
df['CitRevNum'] = CitRevNum

# Просматриваем корреляцию с рэнкингом и рейтингом 
import numpy as np
kr1 = np.corrcoef(df['CitRevNum'], df['Ranking'])
kr2 = np.corrcoef(df['CitRevNum'], df['Rating'])
# kr3 = np.corrcoef(df['Rev/Pop'], df['Ranking'])

print(kr1[0,1],kr2[0,1])

0.6586795123977813 -0.033775458942921215


In [74]:
# Добавляем количество ресторанов в городе
QID = [1 for x in range(len(df['City']))]
df['QID'] = QID
DDF = df.groupby('City').sum()
QR = DDF['QID']
CRQ = []
for i in range(0,len(df['City'])):
    CRQ.append(QR[df['City'][i]])
df['CRQ'] = CRQ
mr1 = np.corrcoef(df['CRQ'], df['Ranking']) # 0.7091149958375524
mr2 = np.corrcoef(df['CRQ'], df['Rating']) # -0.061376216928803094
# print(mr1[0,1],mr2[0,1])
# Добавим отношение отзывов к количеству ресторанов
df['RevpRest'] = df['Number of Reviews']/df['CRQ']


In [75]:
import re

# Используем регулярные выражения, чтобы "вытащить" даты из 'Reviews'
pattern = re.compile('[0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9]')
DatDict = []
DatNorm = []
qZero = 0
qUno = 0
m = []

def dp(Uno, Dos):
    import datetime
    FDate = datetime.date(int(Uno[-4:]),int(Uno[:2]),int(Uno[3:5]))
    SDate = datetime.date(int(Dos[-4:]),int(Dos[:2]),int(Dos[3:5]))
    TDate = abs((SDate - FDate).days)
    return TDate

for i in range(0,len(df['Reviews'])):
    m = pattern.findall(df['Reviews'][i])
    if len(m) == 0:
        qZero += 1
        DatDict.append(0) 
    elif len(m) == 1:
        qUno += 1
        DatDict.append(141.7)
    else:
        DatDict.append(dp(m[0], m[1]))
        DatNorm.append(dp(m[0], m[1]))

df['DifDat'] = DatDict

In [114]:
# Запускаем механизм обучения и считаем новый MAE

In [83]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['City', 'Cuisine Style', 'Reviews', 'URL_TA', 'ID_TA', 'Rating'], axis = 1)
y = df['Rating']

# Уточняем X: убираем нечисловые данные 
X = df.drop(['City', 'Cuisine Style', 'Reviews', 'URL_TA', 'ID_TA','Rating','QID', 'Number of Reviews'], axis = 1)
# Заполняет пропуски нулями
X=X.fillna(0)

In [88]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [89]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [90]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [91]:
# Создаём модель
import numpy as np
regr = RandomForestRegressor(n_estimators=100)
# X_train
# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [92]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.20851050000000002
